# 패키지 import & setting

In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

import time
from bs4 import BeautifulSoup

import re

import pandas as pd
import numpy as np

In [17]:
# 웹 드라이버 설정
## Chrome 옵션 설정
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("--headless")  # 브라우저를 백그라운드에서 실행
chrome_options.add_argument("--lang=ko")

## 웹 드라이버 설정 및 실행
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

# 구글 플레이스토어 리뷰 크롤링

In [53]:
# Chrome 옵션 설정 (한국어로 설정)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--lang=ko")

# 웹 드라이버 설정 및 실행
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

try:
    # 1. URL 설정
    url = 'https://play.google.com/store/apps/details?id=com.elevenst&hl=ko'
    driver.get(url)
    
    # 2. 평가 및 리뷰 페이지 전환
    ## 버튼 로딩 대기
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#yDmH0d > c-wiz.SSPGKf.Czez9d > div > div > div:nth-child(1) > div > div.wkMJlb.YWi3ub > div > div.qZmL0 > div:nth-child(1) > c-wiz:nth-child(5) > section > header > div > div:nth-child(2) > button'))
    )
    ## 전환 버튼 클릭
    driver.find_element(By.CSS_SELECTOR, '#yDmH0d > c-wiz.SSPGKf.Czez9d > div > div > div:nth-child(1) > div > div.wkMJlb.YWi3ub > div > div.qZmL0 > div:nth-child(1) > c-wiz:nth-child(5) > section > header > div > div:nth-child(2) > button').click()

    # 3. Review 가져오기
    # 스크롤을 반복적으로 내려서 모든 리뷰 로드
    review_container = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#yDmH0d > div.VfPpkd-Sx9Kwc.cC1eCc.UDxLd.PzCPDd.HQdjr.VfPpkd-Sx9Kwc-OWXEXe-FNFY6c > div.VfPpkd-wzTsW > div > div > div > div > div.fysCi.Vk3ZVd'))
    )
    last_height = driver.execute_script("return arguments[0].scrollHeight", review_container)
    print(last_height)
    i=0
    for i in range(50):
        driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", review_container)
        time.sleep(2)  # 페이지 로딩 대기

        new_height = driver.execute_script("return arguments[0].scrollHeight", review_container)
        last_height = new_height
        i+=1

    # 페이지 소스 가져오기
    page_source = driver.page_source
finally:
    # 브라우저 닫기
    # driver.quit()
    print('done')

8359
done


In [54]:
# BeautifulSoupt으로 HTML 파싱
soup = BeautifulSoup(page_source, 'html.parser')
reviews = soup.find_all('div', 'RHo1pe')

review_data = []
for review in reviews:
    # 리뷰 이름, 날짜, 별점, 텍스트 추출
    reviewer_name = review.find('div', attrs={'class':'X5PpBb'}).text
    review_date = review.find('span', attrs={'class':'bp9Aid'}).text
    review_score = review.find('div', attrs={'role':'img'})['aria-label']
    review_text = review.find('div', attrs={'class':'h3YV2d'}).text

    # 텍스트에서 별점만 추출
    score_num = re.search(r'만점에 (\d+)개를 받았습니다', review_score).group(1)
   
    # review_data에 저장
    review_data.append({
        'name': reviewer_name,
        'date': review_date,
        'score': score_num,
        'review': review_text
    })

In [55]:
# 리뷰 데이터 출력 (확인용)
for review in review_data:
    print(review)
    print()

{'name': '남상윤', 'date': '2024년 6월 17일', 'score': '4', 'review': '오랜 단골입니다만,타사이트에 비해 변화와 혁신의 모습이 없어 보여 별하나 뺍니다. 11번가의 장점이었던 다양한 쿠폰과 할인,적립의 메리트가 점점 줄어 평이하다 못해 잘 찾지 않는 플랫폼이 되어 가는 듯해 안타깝습니다. 저 역시 30대초반부터 단골이지만, 최근에는 쿠팡,네이버페이 등 주문 빈도가 높습니다. 최초의 11번가 다운 소비자 접근방법이 필요하다 여깁니다.'}

{'name': '신동윤 (돌쇠)', 'date': '2024년 5월 18일', 'score': '5', 'review': '11번가와 판매자님들께서 가성비 좋은 품질에 고객 맞춤형 서비스를 제공하니 이용을 안할수가 없네요 ㅎㅎ 11번가든 판매자님이든 초심을 잃지 않는 것이야말로 무한경쟁 온라인 쇼핑몰에 대한 고객의 신뢰를 다시 회복하는게 아닐까 생각합니다^^ 우리 토종 쇼핑몰 11번가가 나날이 번창하길 바랍니다~~^^ 제가 싼 맛에 중국 직구 쇼핑몰들! **나, 테*도, 많이 이용을 해봤는데, 의류,가정, 생활,주방용품 많큼은 두세번 입고, 직접 사용을 해보니, 품질이 죄다 다 쓰레기 같아 옷은 입지도 못하고 재활용장에 기부 처리 했습니다!'}

{'name': '이효경', 'date': '2024년 4월 25일', 'score': '2', 'review': '음...대체적으로는 맘에드는상품도있지만...정말로 맘에안드는상품들도있습니다!! 근데..매일출석게임해서(고양기키우기)가 변경이돼었는데..예전보다 클러버보다 넘~~~어렵고 짜증이 납니다!! 세번 먹이받는시간두....예매하고 차라히..예전처럼 11개가 모이면 상품이오는걸로 다시했으면 좋겠네요(저는 그런맛으로 즐겼으니까요)!! 근데..지금은 점점 하기도싫어지고 지쳐갑니다!! 저렇게해서 8단계까지 상품을 받는다면...안하는게 맞네요!! 저만 그런가요??'}

{'name': '김서희', 'date': '2023년 12월 22일', '

In [57]:
# 데이터 프레임으로 변환 
review_df = pd.DataFrame(review_data)

# score 컬럼: to integer
review_df['score'] = review_df['score'].apply(lambda x: int(x))

# date 컬럼: to datetime
review_df['date'] = pd.to_datetime(review_df['date'], format='%Y년 %m월 %d일')

In [58]:
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [59]:
review_df

,name,date,score,review
0,남상윤,2024-06-17,4,"오랜 단골입니다만,타사이트에 비해 변화와 혁신의 모습이 없어 보여 별하나 뺍니다. ..."
1,신동윤 (돌쇠),2024-05-18,5,11번가와 판매자님들께서 가성비 좋은 품질에 고객 맞춤형 서비스를 제공하니 이용을 ...
2,이효경,2024-04-25,2,음...대체적으로는 맘에드는상품도있지만...정말로 맘에안드는상품들도있습니다!! 근데...
3,김서희,2023-12-22,5,SKT 사용자라 11절 T멤버십 추가 할인 우주패스 같은 연계 혜택이 있고 11페이...
4,ye kim,2023-12-13,5,필요한 물건이 있으면 가장 먼저 11번가에 검색해 볼 정도로 애용하고 있는 학생입니...
...,...,...,...,...
1015,송천동고선생,2023-12-13,5,사고 싶은 상품을 찾기 쉽게 구분해놓고 여러 할인 혜택이 많아 자주 사용 중입니다....
1016,신민호,2024-04-18,5,빠른배송 및 제품에 다양성
1017,길기남,2024-04-21,5,제품구매시 가격비교등 편한부분이 많네요
1018,신민재,2024-04-24,5,조립해서 사용하기 좋고 빠리 왔응션 좋겠음
